In [ ]:
import whisper
from pytube import YouTube
import requests
import base64
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
from typing import List
import codecs
import nltk
from pydub import AudioSegment
import os
import io
import math
import requests
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"
openai.api_key = ""

In [ ]:
import nltk
#download punkt package
nltk.download()

In [ ]:
def youtube_to_transcript(link):
    model = whisper.load_model("large")
    yt = YouTube(link)
    path = yt.streams.filter(only_audio=True)[0].download(filename="audio.mp4")
    options = whisper.DecodingOptions(without_timestamps=True)
    transcription = model.transcribe(path)
    return transcription["text"]

In [ ]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> List[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]
def vector_similarity(x: List[float], y: List[float]) -> float:
    return np.dot(np.array(x), np.array(y))

In [ ]:
def getchunks(string):
    tokens = nltk.word_tokenize(string)
    num_tokens = len(tokens)
    print(num_tokens)
    if num_tokens <= 3000:
        return [string]
    max_chunk_size = 2500
    chunks = []
    start = 0
    while start < num_tokens:
        end = start + max_chunk_size
        if end >= num_tokens:
            end = num_tokens
       
        chunks.append(' '.join(tokens[start:end]))
        
        start = end
        
    return chunks

In [ ]:
def getresponse(context,query):
    prompt = f"Answer the question as concisely as possible using the provided context:. question: {query} context:{context}"
    params = {
        "engine": "text-davinci-003",
        "prompt": prompt,
        "temperature": 0.0,
        "max_tokens": 500,
    }
    response = openai.Completion.create(**params)
    return response["choices"][0]["text"].strip(" \n")

In [ ]:
def final_wrapper(transcript,query):
    documents=getchunks(transcript)#returns list of chunks
    document_embed=[]
    for i in documents:
        document_embed.append(get_embedding(i))
    query_embed=get_embedding(query)
    similarity=[]
    for i in document_embed:
        similarity.append(vector_similarity(i,query_embed))
    max_index = similarity.index(max(similarity))
    context=documents[max_index]
    return getresponse(context,query)

In [ ]:
#insert a youtube link and your query here
link=""
query=""

In [ ]:
final_wrapper(youtube_to_transcript(link),query)